In this program, we will build the classifier to be used for the visualization.

In [44]:
import pandas as pd
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [58]:
test= pd.read_csv("test.csv")
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv',header=None)

In [60]:
t = test.drop(columns=["Player", "Ballot"])

In [61]:
x_train.era=x_train.era.astype(str)
t.era=t.era.astype(str)

In [62]:
# normalize training data
maxes=[]
mins=[]
valid = x_train[x_train.G>30]
for key in x_train.columns[:-2]:    
    mx = max(valid[key])
    mn = min(valid[key])
    maxes.append(mx)
    mins.append(mn)
    x_train[key]=(x_train[key]-mn)/(mx-mn)

In [64]:
# normalize test data
i=0
for key in t.columns[:-2]:    
    mx=maxes[i]
    mn=mins[i]
    t[key]=(t[key]-mn)/(mx-mn)
    i+=1

In [65]:
# one hot encode categorical data
t.era=pd.Categorical(t.era, categories=x_train.era.unique(),ordered=False)
x_train = pd.get_dummies(x_train)
t = pd.get_dummies(t)


y_test= test.Ballot
x_test = t

In [70]:
# build classifier 
model = Sequential()

model.add(Dense(
    units              = 23, 
    activation         = "relu", 
    input_dim        = 23))

  # Dropout to prevent overfitting
model.add(Dropout(rate=0.4))

model.add(Dense(
    units              = 23, 
    activation         = "relu"))


#   # Dropout to prevent overfitting
model.add(Dropout(rate=0.2))

model.add(Dense(
    units              = 1, 
    activation         = "sigmoid"))


model.compile(
    optimizer = 'adam',
    loss      = 'binary_crossentropy',
    metrics   = ['accuracy']
  )



In [71]:
# fit model
model.fit(x_train, y_train, batch_size=1, nb_epoch=4,verbose=1, validation_data=(x_test, y_test) )

W0628 14:30:15.581768 139662613509952 training.py:593] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 15117 samples, validate on 1106 samples
Epoch 1/4
15117/15117 [==============================] - 24s 2ms/sample - loss: 0.0566 - acc: 0.9877 - val_loss: nan - val_acc: 0.9910
Epoch 2/4
15117/15117 [==============================] - 23s 2ms/sample - loss: 0.0298 - acc: 0.9889 - val_loss: nan - val_acc: 0.9910
Epoch 3/4
15117/15117 [==============================] - 24s 2ms/sample - loss: 0.0262 - acc: 0.9891 - val_loss: nan - val_acc: 0.9937
Epoch 4/4
15117/15117 [==============================] - 25s 2ms/sample - loss: 0.0254 - acc: 0.9901 - val_loss: nan - val_acc: 0.9928


In [99]:
# convert model to tensorflowjs 
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '../MVP-predictor/')